## Working with Transactions and the DBAPI

With the `Engine` object ready to go, we may now proceed to dive into the basic operation of an `Engine` and its primary interactive endpoints, the `Connection` and `Result`. We will additionally introduce the `ORM`'s facade for these objects, known as the `Session`.

> When using the ORM, the `Engine` is managed by another object called the `Session`. The `Session` in modern SQLAlchemy emphasizes a transactional and SQL execution pattern that is largely identical to that of the `Connection` discussed below, so while this subsection is Core-centric, all of the concepts here are essentially relevant to ORM use as well and is recommended for all ORM learners. The execution pattern used by the `Connection` will be contrasted with that of the `Session` at the end of this section.

#### Getting a Connection

The sole purpose of the `Engine` object from a user-facing perspective is to provide a unit of connectivity to the database called the `Connection`. When working with the Core directly, the `Connection` object is how all interaction with the database is done. As the `Connection` represents an open resource against the database, we want to always limit the scope of our use of this object to a specific context, and the best way to do that is by using `Python context manager` form, also known as the `with statement`.

In [1]:
from sqlalchemy import create_engine, text

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
with engine.connect() as conn:
    result = conn.execute(text("select 'Hello World!'"))
    print(result.all())

2022-09-19 06:13:46,656 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-19 06:13:46,658 INFO sqlalchemy.engine.Engine select 'Hello World!'
2022-09-19 06:13:46,660 INFO sqlalchemy.engine.Engine [generated in 0.00445s] ()
[('Hello World!',)]
2022-09-19 06:13:46,663 INFO sqlalchemy.engine.Engine ROLLBACK
